<a href="https://colab.research.google.com/github/robonesky/oreilly/blob/main/RM%20Lab%20Notes%209%20202301%20cohort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('seaborn')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet

from sklearn.metrics import mean_squared_error

pd.options.display.float_format = '{:,.2f}'.format

%pip install watermark

from watermark import watermark
%load_ext watermark
%watermark -a "Rob Onesky" -u -d -v -m -iv

import warnings
warnings.filterwarnings('ignore')

<ipython-input-4-503726bdd91e>:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.2 MB/s eta 0:00:00
Author: Rob Onesky

Last updated: 2023-06-13

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

Compiler    : GCC 9.4.0
OS          : Linux
Release     : 5.15.107+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

pandas    : 1.5.3
seaborn   : 0.12.2
numpy     : 1.22.4
matplotlib: 3.7.1



In [6]:
df = pd.read_csv(
    'https://raw.githubusercontent.com/kannansingaravelu/datasets/main/niftyindex.csv',
    index_col = 0#,
    #dayFirst = True
)

df

,Open,High,Low,Close,Volume
02/01/2012,"4,640.20","4,645.95","4,588.05","4,636.75",108461000
03/01/2012,"4,675.80","4,773.10","4,675.80","4,765.30",146621008
04/01/2012,"4,774.95","4,782.85","4,728.85","4,749.65",165939008
05/01/2012,"4,749.00","4,779.80","4,730.15","4,749.95",177863008
06/01/2012,"4,724.15","4,794.90","4,686.85","4,754.10",176057008
...,...,...,...,...,...
26/12/2022,"17,830.40","18,084.10","17,774.25","18,014.60",176652336
27/12/2022,"18,089.80","18,149.25","17,967.45","18,132.30",214257088
28/12/2022,"18,084.75","18,173.10","18,068.35","18,122.50",193873168
29/12/2022,"18,045.70","18,229.70","17,992.80","18,191.00",281052832


In [7]:
df.describe()

,Open,High,Low,Close,Volume
count,"2,719.00","2,719.00","2,719.00","2,719.00","2,719.00"
mean,"10,124.90","10,175.29","10,057.47","10,117.57","289,222,065.64"
std,"3,709.61","3,725.53","3,685.31","3,706.95","195,276,778.85"
min,"4,640.20","4,645.95","4,588.05","4,636.75","2,826,000.00"
25%,"7,637.00","7,681.00","7,581.65","7,623.80","159,178,008.00"
50%,"9,376.95","9,431.90","9,301.35","9,351.85","219,496,064.00"
75%,"11,700.85","11,755.12","11,631.33","11,684.00","340,908,832.00"
max,"18,871.95","18,887.60","18,778.20","18,812.50","1,810,971,008.00"


In [9]:
df.isnull().sum()

Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64

In [13]:


# create features
def create_features(frame):
    df = frame.copy()
    multiplier = 2

    # spy features
    df['OC'] = df['Close'] / df['Open'] - 1
    df['HC'] = df['High'] / df['Low'] - 1
    df['GAP'] = df['Open'] / df['Close'].shift(1) - 1
    df['RET'] = np.log(df['Close'] / df['Close'].shift(1))

    for i in [3]:
        df['RET' + str(i)] = df['RET'].rolling(i).sum()
        df['MA' + str(i)] = df['Close'] / df['Close'].rolling(i).mean() 
        df['OC'+ str(i)] = df['OC'].rolling(i).mean() 
        df['HC'+ str(i)] = df['HC'].rolling(i).mean() 
        df['GAP'+ str(i)] = df['GAP'].rolling(i).mean()
        df['STD'+ str(i)] = df['RET'].rolling(i).std()
        df['UB'+str(i)] = df['Close'].rolling(i).mean() + df['Close'].rolling(i).std() * multiplier
        df['LB'+str(i)] = df['Close'].rolling(i).mean() - df['Close'].rolling(i).std() * multiplier
    
    df['Label'] = df['Close'].shift(-1)
    
    # drop NaN values
    df.dropna(inplace=True)
    return df



  

In [14]:
df = create_features(df)
display(df.shape)

(2711, 18)